# Field Validation

In [2]:

import pandas as pd #reading dataset
import matplotlib as mpl    #plotting, visualization
import matplotlib.pyplot as plt
import tensorflow as tf #building model
from tensorflow.keras.models import Model
import numpy as np  #playing output with model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import sys
sys.path.append('../../')   # Add parent directory to Python path
from utils.Preprocessing import *
from utils.Segmentation import *


In [2]:
#load data
df_data = pd.read_csv('../../data/test/Accelerometer_ziyi.csv') 
df_data

,Date,NTP,GNSS-Time,Acc-X,Acc-Y,Acc-Z
0,2025-03-28T16:00:44.931,2025-03-28 16:00:48.289,1743174048272,1.781281,4.735733,8.374924
1,2025-03-28T16:00:44.935,2025-03-28 16:00:48.293,1743174048276,2.643204,5.118805,8.274368
2,2025-03-28T16:00:44.942,2025-03-28 16:00:48.300,1743174048285,2.097321,5.037399,7.814682
3,2025-03-28T16:00:44.945,2025-03-28 16:00:48.303,1743174048286,1.891418,5.061340,8.034943
4,2025-03-28T16:00:44.951,2025-03-28 16:00:48.308,1743174048292,2.892197,5.272034,7.891296
...,...,...,...,...,...,...
268724,2025-03-28T16:22:59.273,2025-03-28 16:23:02.631,1743175382614,2.863464,5.655106,7.546524
268725,2025-03-28T16:22:59.277,2025-03-28 16:23:02.636,1743175382619,2.145203,5.549759,7.067688
268726,2025-03-28T16:22:59.284,2025-03-28 16:23:02.642,1743175382625,2.815582,5.281616,8.034943
268727,2025-03-28T16:22:59.288,2025-03-28 16:23:02.646,1743175382629,2.987961,5.559341,8.226471


In [3]:
df_curb = pd.read_csv('../../data/test/ESP_curb.csv')
df_curb

,Date,NTP,GNSS-Time,Value
0,2025-03-28T16:00:59.302,2025-03-28 16:01:02.660,1743174062643,Curb Up
1,2025-03-28T16:01:38.562,2025-03-28 16:01:41.920,1743174101903,Curb Up
2,2025-03-28T16:01:49.559,2025-03-28 16:01:52.917,1743174112903,Curb Up
3,2025-03-28T16:01:56.627,2025-03-28 16:01:59.985,1743174119969,Curb Up
4,2025-03-28T16:02:06.915,2025-03-28 16:02:10.274,1743174130257,Curb Up
...,...,...,...,...
135,2025-03-28T16:22:22.830,2025-03-28 16:22:26.188,1743175346171,Curb Up
136,2025-03-28T16:22:28.231,2025-03-28 16:22:31.589,1743175351572,Curb Up
137,2025-03-28T16:22:30.179,2025-03-28 16:22:33.537,1743175353520,Curb Up
138,2025-03-28T16:22:42.238,2025-03-28 16:22:45.596,1743175365579,Curb Up


In [4]:
#missing value
missing_info = handle_missing_values_length_index(df_data)
print(missing_info)
output = '../../data/test/Accelerometer_ziyi_filled_missing_values.csv'
fill_missing_values(df_data,output)
# check missing value
df_data.isnull().sum() 

No missing values longer than 100
{}


Date         0
NTP          0
GNSS-Time    0
Acc-X        0
Acc-Y        0
Acc-Z        0
dtype: int64

In [5]:
def label_curb_scenes(df_data, df_curb, window_size=100):
    """
    Labels curb scenes in the data and extends labels to surrounding data points.
    
    Args:
        df_data (pd.DataFrame): Original dataframe containing NTP and Acc-Z data
        df_curb (pd.DataFrame): Dataframe containing curb timestamps
        window_size (int): Number of data points to label before and after each curb scene
        
    Returns:
        pd.DataFrame: DataFrame with labeled curb scenes
    """
    # Create copy with selected columns
    df_selected = df_data[['NTP', 'Acc-Z']].copy()
    df_selected['curb_scene'] = 0
    
    # Convert NTP columns to datetime
    df_selected['NTP'] = pd.to_datetime(df_selected['NTP'])
    df_curb['NTP'] = pd.to_datetime(df_curb['NTP'])
    
    # Find matching timestamps and label curb scenes
    df_selected.loc[df_selected['NTP'].isin(df_curb['NTP']), 'curb_scene'] = 1
    
    # Get indices where curb_scene is 1
    curb_indices = df_selected.index[df_selected['curb_scene'] == 1].tolist()
    
    # Extend labels to surrounding data points
    for idx in curb_indices:
        start_idx = max(0, idx - window_size)
        end_idx = min(len(df_selected) - 1, idx + window_size)
        df_selected.loc[start_idx:end_idx, 'curb_scene'] = 1
    
    return df_selected

In [ ]:

df_selected = label_curb_scenes(df_data, df_curb)
print(df_selected.shape)
curb_indices = df_selected.index[df_selected['curb_scene'] == 1].tolist()
print(len(curb_indices))

(268729, 3)
5628


In [5]:
#segmentation with index and time
segmented_test_data_with_time_index = segment_acceleration_data_overlapping_count_index(df_selected,0)
segmented_test_data_with_time_index

NameError: name 'df_selected' is not defined

In [12]:
print(segmented_test_data_with_time_index.shape)
curb_indices = segmented_test_data_with_time_index.index[segmented_test_data_with_time_index['curb_scene'] == 1].tolist()
print(len(curb_indices))

(2687, 103)
56


In [13]:
# standardize data
scaler = StandardScaler()
segmented_test_data_scaled = scaler.fit_transform(segmented_test_data_with_time_index.iloc[:, 3:])

In [15]:
# Load the three pre-trained models
import joblib  
autoencoder = tf.keras.models.load_model('../../models/autoencoder_handlebar_best.keras')
lstm_autoencoder = tf.keras.models.load_model('../../models/lstm_autoencoder_handlebar_best.keras')
one_class_svm = joblib.load('../../models/ocsvm_handlebar_best.joblib')

c:\Users\liuzi\miniconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [45]:
print(segmented_test_data_scaled.shape)
# Reshape data for LSTM (add time step dimension)
segmented_test_data_scaled_lstm = np.reshape(segmented_test_data_scaled, 
                       (segmented_test_data_scaled.shape[0], 1, segmented_test_data_scaled.shape[1]))
print(segmented_test_data_scaled_lstm.shape)

(2632, 100)
(2632, 1, 100)


In [46]:
# Make predictions
autoencoder_pred = autoencoder.predict(segmented_test_data_scaled)
lstm_pred = lstm_autoencoder.predict(segmented_test_data_scaled_lstm)
svm_pred = one_class_svm.predict(segmented_test_data_scaled)

83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


c:\Users\liuzi\miniconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneClassSVM was fitted with feature names
  warnings.warn(


In [ ]:
# Load thresholds
autoencoder_threshold = np.load('../../models/autoencoder_threshold.npy')


array(0.6661867, dtype=float32)